# Solving Complex Tasks with A Sequence of Nested Chats

This notebook shows how you can leverage **nested chats** to solve complex task with AutoGen. Nested chats is a sequence of chats created by a receiver agent after receiving a message from a sender agent and finished before the receiver agent replies to this message. Nested chats allow AutoGen agents to use other agents as their inner monologue to accomplish tasks. This abstraction is powerful as it allows you to compose agents in rich ways. This notebook shows how you can nest a pretty complex sequence of chats among _inner_ agents inside an _outer_ agent.

\:\:\:info Requirements

Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).

\:\:\:

In [1]:
import autogen

#config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
#llm_config = {"config_list": config_list}

In [2]:
import os
config_list = [
    {"api_type": "groq", "model": "llama-3.1-70b-versatile", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-70b-8192", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-groq-70b-8192-tool-use-preview", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "mixtral-8x7b-32768", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-groq-8b-8192-tool-use-preview", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "gemma2-9b-it", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama-3.1-8b-instant", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama-guard-3-8b", "api_key": os.environ.get("GROQ_API_KEY")},
    {"api_type": "groq", "model": "llama3-8b-8192", "api_key": os.environ.get("GROQ_API_KEY")},
    ]

llm_config={"config_list" : config_list}

\:\:\:tip

Learn more about the various ways to configure LLM endpoints [here](/docs/topics/llm_configuration).

\:\:\:

### Example Task

Suppose we want the agents to complete the following sequence of tasks:

In [3]:
tasks = [
    """On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.""",
    """Make a pleasant joke about it.""",
]

Since the first task could be complex to solve, lets construct new agents that can serve as an inner monologue.

### Step 1. Define Agents

#### A Group Chat for Inner Monologue
Below, we construct a group chat manager which manages an `inner_assistant` agent and an `inner_code_interpreter` agent.
Later we will use this group chat inside another agent.


In [4]:
inner_assistant = autogen.AssistantAgent(
    "Inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

inner_code_interpreter = autogen.UserProxyAgent(
    "Inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[inner_assistant, inner_code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

#### Inner- and Outer-Level Individual Agents

Now we will construct a number of individual agents that will assume role of outer and inner agents.

In [5]:
assistant_1 = autogen.AssistantAgent(
    name="Assistant_1",
    llm_config={"config_list": config_list},
)

assistant_2 = autogen.AssistantAgent(
    name="Assistant_2",
    llm_config={"config_list": config_list},
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    """,
)

reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a compliance reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    You must review carefully, identify potential issues, and maintain the integrity of the organization.
    Your role demands fairness, a deep understanding of regulations, and a focus on protecting against
    harm while upholding a culture of responsibility.
    """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

### Step 2: Orchestrate Nested Chats to Solve Tasks

#### Outer Level
In the following code block, at the outer level, we have communication between:

- `user` - `assistant_1` for solving the first task, i.e., `tasks[0]`.
- `user` - `assistant_2` for solving the second task, i.e., `tasks[1]`.

#### Inner Level (Nested Chats)
Since the first task is quite complicated, we created a sequence of _nested chats_ as the inner monologue of Assistant_1.

1. `assistant_1` - `manager`: This chat intends to delegate the task received by Assistant_1 to the Manager to solve.

2. `assistant_1` - `writer`: This chat takes the output from Nested Chat 1, i.e., Assistant_1 vs. Manager, and lets the Writer polish the content to make an engaging and nicely formatted blog post, which is realized through the writing_message function.

3. `assistant_1` - `reviewer`: This chat takes the output from Nested Chat 2 and intends to let the Reviewer agent review the content from Nested Chat 2.

4. `assistant_1` - `writer`: This chat takes the output from previous nested chats and intends to let the Writer agent finalize a blog post.

The sequence of nested chats can be realized with the `register_nested_chats` function, which allows one to register one or a sequence of chats to a particular agent (in this example, the `assistant_1` agent).

Information about the sequence of chats can be specified in the `chat_queue` argument of the `register_nested_chats` function. The following fields are especially useful:
- `recipient` (required) specifies the nested agent;
- `message` specifies what message to send to the nested recipient agent. In a sequence of nested chats, if the `message` field is not specified, we will use the last message the registering agent received as the initial message in the first chat and will skip any subsequent chat in the queue that does not have the `message` field. You can either provide a string or define a callable that returns a string.
- `summary_method` decides what to get out of the nested chat. You can either select from existing options including `"last_msg"` and `"reflection_with_llm"`, or or define your own way on what to get from the nested chat with a Callable.
- `max_turns` determines how many turns of conversation to have between the concerned agent pairs.

In [6]:
def writing_message(recipient, messages, sender, config):
    return f"Polish the content to make an engaging and nicely formatted blog post. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {"recipient": manager, "summary_method": "reflection_with_llm"},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
    {"recipient": reviewer, "message": "Review the content provided.", "summary_method": "last_msg", "max_turns": 1},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
]
assistant_1.register_nested_chats(
    nested_chat_queue,
    trigger=user,
)
# user.initiate_chat(assistant, message=tasks[0], max_turns=1)

res = user.initiate_chats(
    [
        {"recipient": assistant_1, "message": tasks[0], "max_turns": 1, "summary_method": "last_msg"},
        {"recipient": assistant_2, "message": tasks[1]},
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Assistant_1):

On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Assistant_1 (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

--------------------------------------------------------------------------------

Next speaker: Inner-assistant



/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/agentchat/chat.py:53: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Inner-assistant (to chat_manager):

To find out the days when Microsoft stock was higher than $400, we'll need to collect the historical stock price data. We can use the yfinance library in Python for this.

```python
# filename: get.stock.historical.price.py

import yfinance as yf

def get_historical_price(symbol, start, end):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start, end=end)
    hist.reset_index(inplace=True)
    return hist[['Date', 'Close']]

stock_price = get_historical_price('MSFT', '2024-01-01', '2024-12-31')
over_400 = stock_price[stock_price['Close'] > 400]
print(over_400)
```

Please execute this code and return the output. Please note that the actual output may vary depending on your execution time and the availability of the data.



Once you provide the output, I will comment on the stock performance.

--------------------------------------------------------------------------------

Next speaker: Inner-code-interpreter


>>>>>>>> EXECUTING CODE B

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Inner-code-interpreter (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
                         Date       Close
15  2024-01-24 00:00:00-05:00  400.371857
16  2024-01-25 00:00:00-05:00  402.669312
17  2024-01-26 00:00:00-05:00  401.734406
18  2024-01-29 00:00:00-05:00  407.492920
19  2024-01-30 00:00:00-05:00  406.369049
..                        ...         ...
179 2024-09-18 00:00:00-04:00  430.809998
180 2024-09-19 00:00:00-04:00  438.690002
181 2024-09-20 00:00:00-04:00  435.269989
182 2024-09-23 00:00:00-04:00  433.510010
183 2024-09-24 00:00:00-04:00  429.170013

[159 rows x 2 columns]


--------------------------------------------------------------------------------

Next speaker: Inner-assistant

Inner-assistant (to chat_manager):

The output shows that there were 159 trading days when Microsoft stock was higher than $400 in 2024. The days are from 2024-01-24 to 2024-09-24.

Based on the output, it seems that Microsoft stock has been performing well, especia

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


User (to Assistant_2):

exitcode: 0 (execution succeeded)
Code output: 



--------------------------------------------------------------------------------
Assistant_2 (to User):

It seems like there is an issue with the regular expression used to extract the stock price. The grep command is not able to find the pattern and is returning an empty string.

To get the current stock price of Microsoft, I'll use a different approach. I'll use a Python library that provides an easy-to-use API for getting stock prices.

```python
# filename: microsoft_stock.py
import yfinance as yf

def get_microsoft_stock_price():
    msft = yf.Ticker("MSFT")
    price = msft.info['regularMarketPrice']

    if price is not None:
        print(f"The current stock price of Microsoft is: {price}")
    else:
        print("Failed to fetch the stock price.")

get_microsoft_stock_price()
```

Please note that you'll need to install the yfinance library first by running `pip install yfinance` in your terminal.

Ple

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/autogen/oai/groq.py:286: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


User (to Assistant_2):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "microsoft_stock.py", line 13, in <module>
    get_microsoft_stock_price()
  File "microsoft_stock.py", line 6, in get_microsoft_stock_price
    price = msft.info['regularMarketPrice']
            ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
KeyError: 'regularMarketPrice'


--------------------------------------------------------------------------------
Assistant_2 (to User):

It seems like the 'regularMarketPrice' key is not available in the info dictionary. This could be due to a change in the library or the API it's using.

To get the current stock price of Microsoft, we can try using a different key that is available in the info dictionary.

```python
# filename: microsoft_stock.py
import yfinance as yf

def get_microsoft_stock_price():
    msft = yf.Ticker("MSFT")
    info = msft.info

    if 'currentPrice' in info:
        print(f"The current stock price of Microsoft is: {info['curre